# Hazard Predictions

## The Challenge 

### Automatic Hazard Detection  on Transverse Cirrus Bands (TCB)
#### Hurricanes | Tunderstorms | Atmospheric jets

TCBs are bands of clouds oriented perpendicular to the atmospheric flow in which they are embedded. TCBs are often an indicator of strong turbulence and often associated with severe weather such as hurricanes and thunderstorms or atmospheric jets.

Transverse Cirrus Bands Data: s3://impact-datashare/transverse_bands/
        
        

### Imports

In [ ]:
!conda activate skyfolks-env

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (15, 10)
plt.rcParams["figure.dpi"] = 125
plt.rcParams["font.size"] = 14
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.style.use('ggplot')
sns.set_style("whitegrid", {'axes.grid': False})
plt.rcParams['image.cmap'] = 'gray' # grayscale looks better

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
from skimage.io import imread as imread
from skimage.util import montage
from PIL import Image
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
from skimage.color import label2rgb

###  Reading Data

REUSE dataset generator and augmentations from Examples
Link: https://github.com/nasa/spaceapps-phenomena_detection/blob/dev/examples/ml_model_demo_1.ipynb

Apply the logic to the Hurricane Dataset | Transverse Cirrus Bands (TCB)


This is used to push subsets of input images in memory for Keras so that there is no out of memory error during Training phase

In [ ]:
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)

image_dir = Path('.') / 'data' / 'transverse_bands'
mapping_file = Path('.') / 'data' / 'list_of_images.json'
image_df = pd.read_json(mapping_file)

image_df['is_hurricane'] = image_df['path'].map(lambda x: "yes" in x) 

image_df.sample(3)


In [ ]:
image_df.describe()

In [ ]:
image_df.head()

In [ ]:
print(image_df['path'].map(lambda x: Path(x).exists()).value_counts()) # make sure everything is found

In [ ]:
image_df['is_hurricane'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_df = image_df
train_df.reset_index(inplace=True)
print(train_df.shape[0], 'training images')

In [ ]:
image_df.index

In [ ]:
image_df.insert(0, 'id', image_df.index)

In [ ]:
train_x_vec = np.stack(train_df[['id', 'path']], 0)
train_y_vec = np.stack(train_df['is_hurricane'], 0)
print(train_x_vec.shape, '->', train_y_vec.shape)

In [ ]:
image_df

In [ ]:
from PIL import Image

def get_image_pixels(imagePath):
    im = Image.open(imagePath, 'r')
    width, height = im.size
    pixel_values = list(im.getdata())

    return pixel_values

In [ ]:
image_df['image_rgb'] = [ get_image_pixels(x) for x in image_df['path'] ]

In [ ]:
image_df.head()

### Data Cleaning

- Extract color features 
- reduce color noise
- run PCA to sample the data


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
plt.rcParams["figure.figsize"] = (6, 6)
plt.rcParams["figure.dpi"] = 200
plt.rcParams["font.size"] = 14
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.style.use('ggplot')
sns.set_style("whitegrid", {'axes.grid': False})
plt.rcParams['image.cmap'] = 'viridis'

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
from skimage.io import imread
from skimage.util import montage
from tqdm import tqdm
tqdm.pandas() # hack progressbars into pandas
montage_rgb = lambda x, **kwargs: np.stack([montage(x[:, :, :, i], **kwargs) for i in range(x.shape[3])], -1)

In [ ]:
satellite_dir = Path('data/transverse_bands')
image_df = pd.DataFrame({'path': list(satellite_dir.glob('**/*.jp*g'))})
image_df.sample(3)

In [ ]:
image_df['path'][0].__str__()

#### Generate the variable is_hurricane if "yes" string is in the path

In [ ]:
image_df['is_hurricane'] = image_df['path'].map(lambda x: "yes" in x.__str__()) 
image_df.sample(3)

### Downgrading the IMAGE color spectrum

- Currently have 2^8 | 8-bit  and 3 channel| Red, Green, Blue
- Convert it to a 8-bit format to reduce the colors

In [ ]:
test_image = Image.open(image_df['path'].iloc[2739]) # normal image
# convert to 8bit color (animated GIF) and then back
web_image = test_image.convert('P', palette='WEB', dither=None)
few_color_image = web_image.convert('RGB')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.imshow(test_image)
ax2.imshow(few_color_image)

In [ ]:
print('Unique colors before', len(set([tuple(rgb) for rgb in np.array(test_image).reshape((-1, 3))])))
print('Unique colors after', len(set([tuple(rgb) for rgb in np.array(few_color_image).reshape((-1, 3))])))

### Take a look at the RGBs patterns

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))
for c_channel, c_name in enumerate(['red', 'green', 'blue']):
    ax1.hist(np.array(test_image)[:, :, c_channel].ravel(), 
             color=c_name[0], 
             label=c_name, 
             bins=np.arange(256), 
             alpha=0.5)
    ax2.hist(np.array(few_color_image)[:, :, c_channel].ravel(), 
             color=c_name[0], 
             label=c_name, 
             bins=np.arange(256), 
             alpha=0.5)

In [ ]:
idx_to_color = np.array(web_image.getpalette()).reshape((-1, 3))/255.0

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))
ax1.imshow(few_color_image)
counts, bins = np.histogram(web_image, bins=np.arange(256))
for i in range(counts.shape[0]):
    ax2.bar(bins[i], counts[i], color=idx_to_color[i])
ax2.set_yscale('log')
ax2.set_xlabel('Color Id')
ax2.set_ylabel('Pixel Count')

In [16]:
def color_count_feature(in_path):
    raw_image = Image.open(in_path) 
    web_image = raw_image.convert('P', palette='WEB', dither=None)
    counts, bins = np.histogram(np.array(web_image).ravel(), bins=np.arange(256))
    return counts*1.0/np.prod(web_image.size) # normalize output

### Process the images and extract color features

In [ ]:
%%time
image_df['color_features'] = image_df['path'].progress_map(color_count_feature)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 10))
combined_features = np.stack(image_df['color_features'].values, 0)
ax1.imshow(combined_features)
ax1.set_title('Raw Color Counts')
ax1.set_xlabel('Color')
ax1.set_ylabel('Frequency')
ax1.set_aspect(0.01)
color_wise_average = np.tile(np.mean(combined_features, 0, keepdims=True), (combined_features.shape[0], 1)).clip(1/(128*128), 1)
ax2.imshow(combined_features/color_wise_average, vmin=0.05, vmax=20)
ax2.set_title('Normalized Color Counts')
ax2.set_xlabel('Color')
ax2.set_ylabel('Frequency')
ax2.set_aspect(0.01)


# Run PCA on the combined features
- Helps visualize the data
- Dimension reduction

In [ ]:
from sklearn.decomposition import PCA
xy_pca = PCA(n_components=2)
xy_coords = xy_pca.fit_transform(combined_features)
image_df['x'] = xy_coords[:, 0]
image_df['y'] = xy_coords[:, 1]

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(15, 15))
for c_group, c_row in image_df.groupby('is_hurricane'):
    ax1.plot(c_row['x'], c_row['y'], '*', label=c_group)
ax1.legend()

#### Show images grouped by color similarity
- based on the principal components return by PCA

In [ ]:
def show_xy_images(in_df, image_zoom=1):
    fig, ax1 = plt.subplots(1,1, figsize=(10, 10))
    artists = []
    for _, c_row in in_df.iterrows():
        c_img = Image.open(c_row['path']).resize((64, 64))
        img = OffsetImage(c_img, zoom=image_zoom)
        ab = AnnotationBbox(img, (c_row['x'], c_row['y']), xycoords='data', frameon=False)
        artists.append(ax1.add_artist(ab))
    ax1.update_datalim(in_df[['x', 'y']])
    ax1.autoscale()
    ax1.axis('off')
show_xy_images(image_df.sample(200))

#### Save Extracted Features

In [ ]:
image_df['path'] = image_df['path'].map(str) # saving pathlib objects causes problems
image_df.to_json('color_features.json')

In [ ]:
image_df.sample(3)

In [ ]:
image_df['data_split'] = 'train_another'
image_df['data_split'].value_counts()

### Split data into test/train

In [ ]:
from sklearn.model_selection import train_test_split
train_df = image_df.query('data_split=="train_another"')
train_df.reset_index(inplace=True)
print(train_df.shape[0], 'training images')

In [ ]:
train_x_vec = np.stack(train_df['color_features'].values, 0)
train_y_vec = np.stack(train_df['is_hurricane'], 0)
print(train_x_vec.shape, '->', train_y_vec.shape)

#### Helper function to show model results

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

def show_model_results(in_model, use_split=None, plot_type='swarm'):
    fig, m_axs = plt.subplots(4, 2, figsize=(15, 30))
    m_axs = m_axs.flatten()
    all_rows = []
    ax1 = m_axs[0]
    
    if use_split is None:
        cur_df = image_df.copy()
    else:
        cur_df = image_df.query('data_split=="{}"'.format(use_split)) 
    
    for c_split, example_df in cur_df.groupby('data_split'):
        example_df = example_df.reset_index()
        x_vec = np.stack(example_df['color_features'].values, 0)
        y_vec = np.stack(example_df['is_hurricane'], 0)

        valid_pred = in_model.predict(x_vec)
        tpr, fpr, _ = roc_curve(y_vec[:], valid_pred[:])
        auc = roc_auc_score(y_vec[:], valid_pred[:])
        acc = accuracy_score(y_vec[:], valid_pred[:]>0.5)
        ax1.plot(tpr, fpr, '.-', label='{}, AUC {:0.2f}, Accuracy: {:2.0%}'.format(c_split, auc, acc))
        all_rows += [pd.DataFrame({'class': y_vec[:], 'prediction': np.clip(valid_pred[:], 0, 1), 'type': 'is_hurricane', 
                                  'split': c_split})]
    
    c_all_df = pd.concat(all_rows)
        
    # show example images
    ax1.legend()
    for (_, c_row), (c_ax) in zip(
        example_df.sample(m_axs.shape[0]).iterrows(), 
                               m_axs[1:-1]):
        
        c_ax.imshow(imread(c_row['path']))
        t_yp = in_model.predict(np.expand_dims(c_row['color_features'], 0))
        c_ax.set_title('Class: {}\n Is Hurricane Prediction: {:2.2%}'.format(c_row['is_hurricane'], t_yp[0]))
        c_ax.axis('off')
        
        t_y = np.array(c_row['is_hurricane'])
    
    # nice dataframe of output
    
    ax1 = m_axs[-1]
    if plot_type=='swarm':
        # prevent overplotting
        sns.swarmplot(data=c_all_df.sample(500) if c_all_df.shape[0]>1000 else c_all_df,
                      hue='class', 
                      y='prediction', 
                      x='type', 
                      size=2.0, 
                      ax=ax1)
    elif plot_type=='box':
        sns.boxplot(data=c_all_df, hue='class', y='prediction', x='type', ax=ax1)
    elif plot_type=='violin':
        sns.violinplot(data=c_all_df, hue='class', y='prediction', x='type', ax=ax1)
    ax1.set_ylim(-0.05, 1.05)
    return c_all_df

# Train Model


#### KNeighbors Regressor 
Nearest Neighbor works by finding the most similar case from the training data using the feature vector. We can directly visualize this by showing which training image was being looked at.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=1)
knn.fit(train_x_vec, train_y_vec)

In [ ]:
show_model_results(knn, None, plot_type='box');

## Dig deeper

In [ ]:
fig, m_axs = plt.subplots(6, 4, figsize=(30, 40))
dummy_web_image = Image.new(size=(1,1), mode='RGB').convert('P', palette='web')

for (c_ax, c_feat_ax, d_ax, d_feat_ax), (_, c_row) in zip(m_axs, 
                            image_df.sample(m_axs.shape[0], random_state=2018).iterrows()):
    
    query_img = Image.open(c_row['path'])
    idx_to_color = np.array(query_img.convert('P', palette='web').getpalette()).reshape((-1, 3))/255.0
    c_ax.imshow(query_img)
    c_ax.set_title(c_row['path'][:25])
    c_ax.axis('off')
    counts, bins = np.histogram(np.ravel(query_img.convert('P', palette='web')), 
                                bins=np.arange(256))
    
    for i in range(counts.shape[0]):
        c_feat_ax.bar(bins[i], counts[i], color=idx_to_color[i], edgecolor='k', linewidth=0.1)
    c_feat_ax.set_yscale('log')
    c_feat_ax.set_xlabel('Color Id')
    c_feat_ax.set_ylabel('Pixel Count')
    c_feat_ax.set_title('Feature Vector')
    
    dist, idx = knn.kneighbors(np.expand_dims(c_row['color_features'], 0))
    m_row = train_df.iloc[idx[0][0]]
    matched_img = Image.open(m_row['path'])
    
    d_ax.imshow(matched_img)
    d_ax.set_title('Closest Match\n{}\nDistance: {:2.1%}'.format(m_row['path'][:25], dist[0][0]))
    d_ax.axis('off')
    
    counts, bins = np.histogram(np.ravel(matched_img.convert('P', palette='web')), 
                                bins=np.arange(256))
    
    for i in range(counts.shape[0]):
        d_feat_ax.bar(bins[i], counts[i], color=idx_to_color[i], edgecolor='k', linewidth=0.1)
    d_feat_ax.set_yscale('log')
    d_feat_ax.set_xlabel('Color Id')
    d_feat_ax.set_ylabel('Pixel Count')
    c_feat_ax.set_title('Matched Feature')

In [ ]:
show_model_results(knn);

#  Try other Model

- RandomForest
- XGBoost

#### Random Forest

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestRegressor

rf_pipe = make_pipeline(RobustScaler(), RandomForestRegressor(n_estimators=200))
rf_pipe.fit(train_x_vec, train_y_vec)
show_model_results(rf_pipe);

#### XGBoost

In [ ]:
from xgboost import XGBRegressor

xg_pipe = make_pipeline(RobustScaler(), 
                        XGBRegressor(objective='reg:linear'))
xg_pipe.fit(train_x_vec, train_y_vec)

In [ ]:
show_model_results(xg_pipe);

###  Test and Predict

In [ ]:
train_x_vec = np.stack(train_df['color_features'].values, 0)

In [ ]:
train_df.head()

In [ ]:
## Get first Element

predictInput = train_df[train_df.index == 5002]
predictInput.head()

In [ ]:
predict_x_vec = np.stack(predictInput['color_features'].values, 0)

### Predict


In [ ]:
print(knn.predict(predict_x_vec));

In [ ]:
print(rf_pipe.predict(predict_x_vec));

In [ ]:
print(xg_pipe.predict(predict_x_vec));

###  Save Model

In [ ]:
import pickle

model_filename = "knn_hurricane.model"

models = {'knn': knn, 'rf_pipe': rf_pipe, 'xg_pipe': xg_pipe}

pickle._dump(models, open(model_filename, 'wb'))

### Reload model

In [ ]:
import pickle 
loaded_classifiers = pickle.load(open(model_filename, 'rb'))

# print(loaded_classifiers['rf_pipe'].predict(predict_x_vec));

# HAZAR-PREDICTION-SERVICE

##### Logic steps

1. Expose Endpoint using Flask

2. Receive Request in JSON

3. Call MODIS to retrieve image for X,Y

4. Run Image by previous Model

5. Return Results in JSON


Contract Request Body:
```json
{
    latitude: 14.021,
    longitude: -124.034,
    predict: [ "hurricane" ]
}
```


#### Flask Endpoint

In [3]:
## Imports
from flask import Flask
from flask import request

In [17]:
def color_count_feature(in_path):
    raw_image = Image.open(in_path) 
    web_image = raw_image.convert('P', palette='WEB', dither=None)
    counts, bins = np.histogram(np.array(web_image).ravel(), bins=np.arange(256))
    return counts*1.0/np.prod(web_image.size) # normalize output

In [25]:
app = Flask(__name__)

@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE')
    return response

@app.route('/hazard/predict', methods=['POST'])
def power():
    if request.headers['Content-Type'] == 'application/json':
        return process_request(request.json)
    else:
        return response("Json record not found in request", 415)

### Start Service



In [ ]:
app.run(host='0.0.0.0', port=8777)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8777/ (Press CTRL+C to quit)
192.168.1.112 - - [04/Oct/2020 14:04:28] "OPTIONS /hazard/predict HTTP/1.1" 200 -


Request received: 
{'latitude': 51.67030573742217, 'longitude': -50.87328749999999, 'predict': ['hurricane']}
Calculating Square Shape...
Shape:  {'lower_latitude': 46.67030573742217, 'left_longitude': -55.87328749999999, 'higher_latitude': 56.67030573742217, 'right_longitude': -45.87328749999999}
Retrieve & download the MODIS image based on (X,Y,Z,K) parameters
1376 2220
Image PATH:  img/modis_hazard_c84abb3c598b408dbc26d548b1d77e6b.png  

Predict if in the image is an Hurricane
img/modis_hazard_c84abb3c598b408dbc26d548b1d77e6b.png
0    img/modis_hazard_c84abb3c598b408dbc26d548b1d77...
Name: path, dtype: object


progress-bar:   0%|                                                                                                                                                    | 0/1 [00:00<?, ?it/s]

[5.74029698e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.14286088e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.06301069e-02 3.88448041e-03 5.23779594e-06 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.27362246e-07
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.48331107e-02 3.37117641e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.04222318e-02 1.28067384e-02 2.12785460e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.94626021e-06 8.21679237e-05
 8.46886130e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000

progress-bar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.38it/s]
192.168.1.112 - - [04/Oct/2020 14:04:37] "POST /hazard/predict HTTP/1.1" 200 -


  index                                               path is_hurricane  \
0   123  img/modis_hazard_c84abb3c598b408dbc26d548b1d77...        False   

                                      color_features data_split  
0  [0.005740296983029541, 0.0, 0.0, 0.0, 0.0, 0.0...    predict  
Probability of hurricane:
[0.11]
Response:  {'latitude': 51.67030573742217, 'longitude': -50.87328749999999, 'predicted': [{'hazardType': 'hurricane', 'probability': 0.11}]}


### curl example to test endpoint


<code>curl -d '{"latitude":-26.055, "longitude":43.273, "predict": [ "hurricane" ]}' -H "Content-Type: application/json" -X POST http://localhost:8777/hazard/predict
</code>


#### Process Request

In [12]:
def process_request(requestJson):
    print("Request received: ")
    print(requestJson)
    
    # Calculate square shape from the X,Y of the click
    print("Calculating Square Shape...")
    squareShape = calculate_square_shape(requestJson['latitude'],requestJson['longitude'])
    print("Shape: ", squareShape)
    
    # Retrieve & download the MODIS image based on (X,Y,Z,K) parameters
    print("Retrieve & download the MODIS image based on (X,Y,Z,K) parameters")
    imagePath = retrieve_modis_image(squareShape['lower_latitude'], squareShape['left_longitude'],
        squareShape['higher_latitude'], squareShape['right_longitude'])
    print("Image PATH: ",imagePath," \n")
    
    # Predict if in the image is an Hurricane    
    print("Predict if in the image is an Hurricane")
    response = predict_is_hurricane(requestJson['latitude'], requestJson['longitude'], imagePath)
    
    # Response
    print("Response: ",response)
    return response
    

#### Calculate Square shape from point lattitudeX, longitudeY

Result Dimensions: [lower_latitude, left_longitude, higher_latitude, right_longitude]


In [5]:

def calculate_square_shape(lattX,longY):
    clickDistanceToEdge = 5; # measured in coordinate degrees
    
    lower_latitude = lattX - clickDistanceToEdge
    left_longitude = longY - clickDistanceToEdge
    higher_latitude = lattX + clickDistanceToEdge
    right_longitude = longY + clickDistanceToEdge
    
    result = {"lower_latitude":lower_latitude, "left_longitude":left_longitude, 
              "higher_latitude":higher_latitude, "right_longitude":right_longitude}
    
    return result
    

### Retrieve MODIS image

In [6]:
la=51.46162974683544
lo=-22.94768591772153

import uuid
uuid.uuid4()

print( str(uuid.uuid4()))

print( uuid.uuid4().hex)

imageTest = "modis_hazard_%s.png" % (uuid.uuid4().hex)

imageTest

1302cae4-39d3-4656-82ad-d8a7306869fc
f2f8d99800c040f4a5b71bb3df970894


'modis_hazard_c6449644896245c980c1c23147844287.png'

In [7]:
import numpy as np
import requests
import uuid

from io import BytesIO
from matplotlib.pyplot import imshow
from PIL import Image

URL = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?SERVICE=WMS&REQUEST=GetMap&layers=MODIS_Aqua_CorrectedReflectance_TrueColor&version=1.3.0&crs=EPSG:4326&transparent=false&width={}&height={}&bbox={}&format=image/tiff&time={}"
KM_PER_DEG_AT_EQ = 111.


# Note: Be careful with the resolution and extent. higher resolution with bigger extent images will be huge
# and probably not load. For the dataset we have 0.25 might be more than enough.
def calculate_width_height(extent, resolution):
    """
    extent: [lower_latitude, left_longitude, higher_latitude, right_longitude],
        EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432]
    resolution: represents the pixel resolution,
        i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5, 1, 5, 10]
    """
    lats = extent[::2]
    lons = extent[1::2]
    km_per_deg_at_lat = KM_PER_DEG_AT_EQ * np.cos(np.pi * np.mean(lats) / 180.)
    width = int((lons[1] - lons[0]) * km_per_deg_at_lat / resolution)
    height = int((lats[1] - lats[0]) * KM_PER_DEG_AT_EQ / resolution)
    print(width, height)
    return width, height


def modis_url(time, extent, resolution):
    """
    time: utc time in iso format EG: 2020-02-19T00:00:00Z
    extent: [lower_latitude, left_longitude, higher_latitude, right_longitude],
        EG: [51.46162974683544,-22.94768591772153,53.03698575949367,-20.952234968354432]
    resolution: represents the pixel resolution,
        i.e. km/pixel. Should be a value from this list: [0.03, 0.06, 0.125, 0.25, 0.5, 1, 5, 10]
    """
    width, height = calculate_width_height(extent, resolution)
    extent = ','.join(map(lambda x: str(x), extent))
    return width, height, URL.format(width, height, extent, time)




def retrieve_modis_image(x,y,z,k):
    width, height, url = modis_url('2020-10-03T00:00:00Z',
                               [x,y,z,k],
                               0.5)
    
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
    #Unique ID generated
    id = uuid.uuid4().hex

    imagePath = "img/modis_hazard_%s.png" % (id)
    
    img.save(imagePath)
    
    return imagePath
    

In [8]:
#Test MODIS
# {'lower_latitude': -56.055, 'left_longitude': 13.273000000000003, 'higher_latitude': 3.9450000000000003, 'right_longitude': 73.273}

retrieve_modis_image(-26.0553, 28.4103, -14.0135, 43.2748)



3100 2673


'img/modis_hazard_9634b8ef7a2d4d18bcda3b036d5eae0d.png'

In [9]:
 # Calculate square shape from the X,Y of the click
print("Calculating Square Shape...")
squareShape = calculate_square_shape(-18.123,35.123)
print("Shape: ", squareShape)

# Retrieve & download the MODIS image based on (X,Y,Z,K) parameters
print("Retrieve & download the MODIS image based on (X,Y,Z,K) parameters")
imagePath = retrieve_modis_image(squareShape['lower_latitude'], squareShape['left_longitude'],
    squareShape['higher_latitude'], squareShape['right_longitude'])
print("Image PATH: ",imagePath," \n")

Calculating Square Shape...
Shape:  {'lower_latitude': -23.123, 'left_longitude': 30.122999999999998, 'higher_latitude': -13.123000000000001, 'right_longitude': 40.123}
Retrieve & download the MODIS image based on (X,Y,Z,K) parameters
2109 2220
Image PATH:  img/modis_hazard_d2f8cfded4c54c45b995fe190f7e2ff5.png  



#### Predict if hurricane 
- Use existing model on the image downloaded
- Decompose the image so that it fits the model input

In [ ]:
train_df.head()

In [ ]:
len(train_df)

In [ ]:
%%time
newId = len(train_df)+1

train_df= train_df.append({'index': newId, 'path':'img/modis_hazard_3e8429697ddd434bb0ad2ddd3063a9d9.png', 'is_hurricane': False, 'data_split':'predict' }, ignore_index=True)

train_df.iloc[-1]


In [ ]:
raw_image = Image.open(train_df.iloc[-1]['path'])
raw_image

In [ ]:
train_df[train_df['index'] == newId]['path']

In [ ]:
newDF = train_df[train_df['index'] == newId]
newDF
newDF['color_features'] = newDF['path'].progress_map(color_count_feature)

In [22]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

def decompose_img(imagePath):
    mock_df = pd.DataFrame( columns=['index', 'path', 'is_hurricane','color_features','data_split'])

    newId = 123
    raw_image = Image.open(imagePath)
    
    mock_df= mock_df.append({'index': newId, 'path':str(imagePath), 'is_hurricane': False, 'data_split':'predict' }, ignore_index=True)

    mock_df.iloc[-1]
    
    newDF = mock_df[mock_df['index'] == newId]
    
    print(np.array(color_count_feature(imagePath)))
#     newDF['color_features'] = np.array(color_count_feature(imagePath))
    newDF['color_features'] = newDF['path'].progress_map(color_count_feature)
    
    return newDF

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [23]:
import pickle

def predict_is_hurricane(latitudeX, longitudeY, imagePath):
    #Load model from saved file
    model_filename = "knn_hurricane.model"
        
    loaded_classifiers = pickle.load(open(model_filename, 'rb'))

    #Decompose image colour features - RGB
    newDF = decompose_img(imagePath)
    print(newDF)
    predict_x_vec = np.stack(newDF['color_features'], 0)
    predicted_probability = loaded_classifiers['rf_pipe'].predict(predict_x_vec)
    
    #Pass the image as input to the model- get probabilty
    print("Probability of hurricane:");
    predicted_probability = loaded_classifiers['rf_pipe'].predict(predict_x_vec)
    print(predicted_probability);
    
    #Format Response
    response = { "latitude": latitudeX, "longitude": longitudeY, "predicted": [ { "hazardType": "hurricane", "probability": predicted_probability[0] }]}
    
    return response

In [24]:
# Test predict function
response = predict_is_hurricane(12.32,123.33,'img/modis_hazard_3e8429697ddd434bb0ad2ddd3063a9d9.png')

progress-bar: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.50it/s]

[13:54:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
img/modis_hazard_3e8429697ddd434bb0ad2ddd3063a9d9.png
0    img/modis_hazard_3e8429697ddd434bb0ad2ddd3063a...
Name: path, dtype: object
[6.57238332e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.45964979e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.21191053e-03 1.03468846e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0

In [ ]:
response